# Predicción de número usando modelo LSTM y landmarks

Este notebook permite seleccionar una imagen, extraer los landmarks con MediaPipe, preprocesarla y predecir el número usando el modelo LSTM entrenado.

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import tensorflow as tf
import joblib
from tkinter import Tk, filedialog

# Cargar modelo y codificador
model = tf.keras.models.load_model("modelo_lstm_landmarks.keras")
le = joblib.load("label_encoder_landmarks.pkl")

## Seleccionar imagen y extraer landmarks

In [ ]:
# Función para extraer landmarks de una imagen
mp_hands = mp.solutions.hands

def extract_landmarks_from_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None, None
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1) as hands:
        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0].landmark
            landmarks_vec = np.array([[l.x, l.y, l.z] for l in hand_landmarks]).flatten()
            return hand_landmarks, landmarks_vec
        else:
            return None, None

In [ ]:
# Seleccionar archivo de imagen
Tk().withdraw()
img_path = filedialog.askopenfilename(title="Selecciona una imagen", filetypes=[("Imágenes", "*.jpg *.jpeg *.png")])
print("Imagen seleccionada:", img_path)

# Verificar si la imagen se puede cargar antes de continuar
img_check = cv2.imread(img_path)
if img_check is None:
    print(f"No se pudo cargar la imagen desde la ruta: {img_path}")
    # Detener ejecución si la imagen no se puede cargar
    raise SystemExit("Ejecución detenida: imagen no válida.")

In [ ]:
# Extraer landmarks y predecir
hand_landmarks, landmarks_vec = extract_landmarks_from_image(img_path)
if landmarks_vec is None:
    print("No se detectó mano en la imagen.")
else:
    # Preparar para LSTM: (1, 21, 3)
    X_pred = landmarks_vec.reshape(1, 21, 3)
    pred = model.predict(X_pred)
    pred_class = le.inverse_transform([np.argmax(pred)])[0]
    print(f"Predicción: {pred_class}")

## Mostrar imagen con puntos clave y predicción

In [ ]:
img = cv2.imread(img_path)
if img is None:
    print(f"No se pudo cargar la imagen desde la ruta: {img_path}")
else:
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(5,5))
    plt.imshow(img_rgb)
    if hand_landmarks is not None:
        mp_drawing = mp.solutions.drawing_utils
        # Re-extract the full HandLandmark object for drawing
        with mp_hands.Hands(static_image_mode=True, max_num_hands=1) as hands:
            results = hands.process(img_rgb)
            if results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(img_rgb, results.multi_hand_landmarks[0], mp_hands.HAND_CONNECTIONS)
        plt.imshow(img_rgb)
        plt.title(f"Predicción: {pred_class}")
    else:
        plt.title("No se detectó mano")
    plt.axis('off')
    plt.show()